# Install and downloads

### Load Huggingface API key
Here we use Huggingface models

Note: Please load a text file that contains your model api key to __Current Folder__.

>Name your file in either ***'api_key_huggingface.txt'***

In [1]:
%%bash
rm -r /content/SearchQuery2FuncCall
git clone https://github.com/XiaoLIUau/SearchQuery2FuncCall.git
pip install --upgrade pip

Cloning into 'SearchQuery2FuncCall'...


In [2]:
%%bash
pip install -r /content/SearchQuery2FuncCall/requirements.txt --quiet
pip install git+https://github.com/huggingface/peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torchdata==0.7.0, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-fdkwhzq1


# Setup

In [1]:
# Setups
from SearchQuery2FuncCall.load_datasets_n_login_HF import q2f_datasets
from SearchQuery2FuncCall.utilities_main import *
from SearchQuery2FuncCall.utilities_train_peft import *

Saved 340 examples to 'q2f_dataset.json'.
Separated 87 Search() examples to 'search_examples.json'.
Separated 253 non-Search() examples to 'non_search_examples.json'.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-be2e7f488507d3dd/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 260
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 80
    })
})
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Base Model and setup model Configs

In [3]:
# Load model function
def load_model(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_use_double_quant=True,
        bnb_8bit_quant_type="nf4",
        bnb_8bit_compute_dtype=torch.bfloat16,
        llm_int8_enable_fp32_cpu_offload=True,
    )
    return AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map='auto',
                    quantization_config=bnb_config,
                    )

# Load model directly
model_name = "atwine/llama-2-7b-chat-fully-quantized-q4-06092023"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
# Original model
original_model = load_model(model_name)

# Check trainable parameters
print('\n### Before freeze all parameters: ###')
print(print_number_of_trainable_model_parameters(original_model))
# Freeze model parameters
freeze_all_parameters(original_model)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at atwine/llama-2-7b-chat-fully-quantized-q4-06092023 and are newly initialized: ['model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.la


### Before freeze all parameters: ###
trainable model parameters: 262410240
all model parameters: 6738415616
percentage of trainable model parameters: 3.89%

### After freeze all parameters: ###
trainable model parameters: 0
all model parameters: 6738415616
percentage of trainable model parameters: 0.00%


# Try with Base Model

In [4]:
example_idxs = [5, 9, 24, 78]#[78]#[49]#
generate_samples(example_idxs,q2f_datasets,original_model,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT:
37.8 celsius to fahrenheit
---------------------------------------------------------------------------------------------------
BASELINE OUTPUT:
UnitConvert(SourceUnit:celsius,TargetUnit:fahrenheit,SourceValue:37.8)
---------------------------------------------------------------------------------------------------
PEFT MODEL GENERATION - OUTPUT:
Calculate(37.8,“°F”)

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT:
Square root of 2.25
---------------------------------------------------------------------------------------------------
BASELINE OUTPUT:
Calculate(sqrt(2.25))
------------------------------------

## PEFT training

In [5]:
# PEFT Setup
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],#, "ffn_kernel"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# peft_model = original_model
peft_model = prepare_model_for_kbit_training(original_model)
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))


########################
# Reduce memory
del original_model
n = gc.collect()
torch.cuda.empty_cache()
########################

output_dir = f'./peft-query-function-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=5,
    logging_steps=20,
    # max_steps=10
)


# reformat dataset for training by adding 'text' variable in examples
q2f_datasets = process_dataset_dict(q2f_datasets, create_text_datasets)
print(q2f_datasets)
# get max sequence length of the 'text' in training dataset
max_seq_length = get_max_seq_length(q2f_datasets)

# Set supervised fine-tuning parameters
peft_trainer = SFTTrainer(
    model=peft_model,
    train_dataset=q2f_datasets['train'],
    # eval_dataset=q2f_datasets['valid'],
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=peft_training_args,
    packing=False,
)

## PEFT training
peft_trainer.train()

## Save PEFT Model
peft_model_path="./peft-query-function-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path,
                                   local_files_only=True
                                   )
tokenizer.save_pretrained(peft_model_path,
                          local_files_only=True
                          )

trainable model parameters: 16777216
all model parameters: 6755192832
percentage of trainable model parameters: 0.25%


Map:   0%|          | 0/260 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'text'],
        num_rows: 260
    })
    test: Dataset({
        features: ['input', 'output', 'text'],
        num_rows: 80
    })
})


Map:   0%|          | 0/260 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
20,0.407700
40,0.058700
60,0.050400
80,0.045000
100,0.042000
120,0.036300
140,0.035000
160,0.033000


('./peft-query-function-checkpoint-local/tokenizer_config.json',
 './peft-query-function-checkpoint-local/special_tokens_map.json',
 './peft-query-function-checkpoint-local/tokenizer.json')

In [6]:
!ls -al ./peft-query-function-checkpoint-local/adapter_model.bin

-rw-r--r-- 1 root root 67201357 Oct 23 06:26 ./peft-query-function-checkpoint-local/adapter_model.bin


In [ ]:
# !zip -r /content/SearchQuery2FuncCall/peft-query-function-checkpoint-local.zip /content/SearchQuery2FuncCall/peft-query-function-checkpoint-local
# from google.colab import files
# files.download("/content/SearchQuery2FuncCall/peft-query-function-checkpoint-local.zip")

In [7]:
!zip -r peft-query-function-checkpoint-local.zip ./peft-query-function-checkpoint-local
from google.colab import files
files.download("./peft-query-function-checkpoint-local.zip")

  adding: peft-query-function-checkpoint-local/ (stored 0%)
  adding: peft-query-function-checkpoint-local/adapter_model.bin (deflated 8%)
  adding: peft-query-function-checkpoint-local/special_tokens_map.json (deflated 73%)
  adding: peft-query-function-checkpoint-local/README.md (deflated 65%)
  adding: peft-query-function-checkpoint-local/tokenizer.json (deflated 74%)
  adding: peft-query-function-checkpoint-local/tokenizer_config.json (deflated 66%)
  adding: peft-query-function-checkpoint-local/adapter_config.json (deflated 46%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load PEFT model  for inference
Require restart runtime due to memory issue

In [8]:
# import os
# os.kill(os.getpid(), 9)

del peft_model
del tokenizer
del peft_trainer
# del original_model

import gc
n = gc.collect()
torch.cuda.empty_cache()

In [9]:
n = gc.collect()
torch.cuda.empty_cache()

In [ ]:
# # Setups
# from SearchQuery2FuncCall.load_datasets_n_login_HF import q2f_datasets
# from SearchQuery2FuncCall.utilities_main import *

In [11]:
### Load PEFT model from previous saved folder
# Note: It will only work if the colab session is not disconnected from previous training session
# Restart the runtime is fine

model_name = "atwine/llama-2-7b-chat-fully-quantized-q4-06092023"
peft_model_base = AutoModelForCausalLM.from_pretrained(model_name,
                                                       torch_dtype=torch.float16,
                                                       device_map="auto")
# peft_model_base = load_model(model_name)
peft_model_path="./peft-query-function-checkpoint-local"#"/content/SearchQuery2FuncCall/peft-query-function-checkpoint-local"#
# Load saved peft trained AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_model_path)#, torch_dtype=torch.bfloat16)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
# Load saved peft trained model
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype=torch.float16,
                                       device_map="auto",
                                       is_trainable=False
                                      )
print('\nBefore merge:')
print(print_number_of_trainable_model_parameters(peft_model))
peft_model = peft_model.merge_and_unload()
print('\nAfter merge:')
print(print_number_of_trainable_model_parameters(peft_model))

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at atwine/llama-2-7b-chat-fully-quantized-q4-06092023 and are newly initialized: ['model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.la

Before merge:
trainable model parameters: 0
all model parameters: 6755192832
percentage of trainable model parameters: 0.00%


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:67: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


After merge:
trainable model parameters: 0
all model parameters: 6738415616
percentage of trainable model parameters: 0.00%


In [12]:
# import shutil
# # Get the path to the folder you want to remove
# folder_path = "/content/Untitled Folder"
# # Remove the folder recursively
# shutil.rmtree(folder_path)
!rm -r ./merged-model
merged_model_path="merged-model"
peft_model.save_pretrained(merged_model_path,local_files_only=True)
tokenizer.save_pretrained(merged_model_path,local_files_only=True)

rm: cannot remove './merged-model': No such file or directory


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1708: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(


('merged-model/tokenizer_config.json',
 'merged-model/special_tokens_map.json',
 'merged-model/tokenizer.json')

In [ ]:
del peft_model_base
del peft_model
del tokenizer

import gc
n = gc.collect()
torch.cuda.empty_cache()

In [20]:
n = gc.collect()
torch.cuda.empty_cache()

In [21]:
merged_model_path="merged-model"
# Load merged peft model in 4 bits
peft_model = AutoModelForCausalLM.from_pretrained(merged_model_path,
                                                  device_map="auto",
                                                  # load_in_8bit=True,
                                                  # torch_dtype=torch.bfloat16
                                                  )
tokenizer = AutoTokenizer.from_pretrained(merged_model_path)#,torch_dtype=torch.bfloat16)

# Generate with selected examples

In [23]:
from datetime import datetime
# record current timestamp
start = datetime.now()
example_indices = [49]#[5, 9, 24, 78]#
generate_samples(example_indices,q2f_datasets,peft_model,tokenizer)
end = datetime.now()
# find difference loop start and end time and display
td = (end - start).total_seconds() * 10**3
print(f"The time of execution of above program is : {td:.03f}ms")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT:
square root of 100
---------------------------------------------------------------------------------------------------
BASELINE OUTPUT:
Calculate(sqrt(100))
---------------------------------------------------------------------------------------------------
PEFT MODEL GENERATION - OUTPUT:
Calculate(sqrt(100))

The time of execution of above program is : 21106.479ms


* 8 bit + torch_dtype=torch.bfloat16
The time of execution of above program is : 22509.366ms
* 8 bit
The time of execution of above program is : 22159.685ms
* 4 bits
The time of execution of above program is : 21559.685ms

# Generate outputs for given test dataset
Here we are using the test dataset

In [24]:
index_s=0 # Start index for example in test dataset
df=generate_dataset(index_s,q2f_datasets,peft_model,tokenizer)
df

0
10
20
30
40
50
60
70


,Inputs,Outputs,Outputs_generated
0,Calculate -5 factorial,Calculate(-5!),Calculate(-5)
1,liters to gallons,"UnitConvert(SourceUnit:liter,TargetUnit:gallon...","UnitConvert(SourceUnit:liter,TargetUnit:gallon..."
2,What is -4.5 times 2,Calculate(-4.5*2),Calculate(-4.5*2)
3,Gallons to liters,"UnitConvert(SourceUnit:gallon,TargetUnit:liter...","UnitConvert(SourceUnit:gallon,TargetUnit:liter..."
4,how to learn a new language,Search(),Search()
...,...,...,...
75,0.125 ounces to grams,"UnitConvert(SourceUnit:ounce,TargetUnit:gram,S...","UnitConvert(SourceUnit:ounce,TargetUnit:gram,S..."
76,convert 2 liters to milliliters,"UnitConvert(SourceUnit:liter,TargetUnit:millil...","UnitConvert(SourceUnit:liter,TargetUnit:millil..."
77,How to cook pasta al dente?,Search(),Search()
78,20 minus 7,Calculate(20-7),Calculate(20-7)


# Evaluate using ROUGE and BLEU scores

In [31]:
evaluate_generations(df['Outputs'],df['Outputs_generated'])

MODEL ROUGE SCORES:
{'rouge1': 0.9827777777777779, 'rouge2': 0.7005208333333333, 'rougeL': 0.9833333333333334, 'rougeLsum': 0.9833333333333332}
MODEL BLEU SCORES:
{'bleu': 0.9334276042273965, 'precisions': [0.9897660818713451, 0.9751655629139073, 0.9580152671755725, 0.9391891891891891], 'brevity_penalty': 0.9669333300994326, 'length_ratio': 0.9674681753889675, 'translation_length': 684, 'reference_length': 707}


## Evaluation Results
MODEL ROUGE SCORES:
{'rouge1': 0.9827777777777779, 'rouge2': 0.7005208333333333, 'rougeL': 0.9833333333333334, 'rougeLsum': 0.9833333333333332}

MODEL BLEU SCORES:
{'bleu': 0.9334276042273965, 'precisions': [0.9897660818713451, 0.9751655629139073, 0.9580152671755725, 0.9391891891891891], 'brevity_penalty': 0.9669333300994326, 'length_ratio': 0.9674681753889675, 'translation_length': 684, 'reference_length': 707}